# Exporting ImageCollections

Earth Engine provides numbers of ways to export `ee.Image` as explained in their [documentation](https://developers.google.com/earth-engine/guides/image_export). `geetools`provides an extention to the `ee.Export` class to export `ee.ImageCollection` as well. This is useful when you have a collection of images and you want to export them all at once.

As the vanilla Earth Engine methods were returning Task objects, these method will return lists of Task objects. This ensures that Once the task are launched they can be fully monitored outside from your initial script.

In [ ]:
import ee, geetools, os, re, httplib2

if "EARTHENGINE_SERVICE_ACCOUNT" in os.environ:
    private_key = os.environ["EARTHENGINE_SERVICE_ACCOUNT"]
    private_key = private_key[1:-1] if re.compile(r"^'[^']*'$").match(private_key) else private_key
    ee.Initialize.geetools.from_service_account(private_key)

elif "EARTHENGINE_PROJECT" in os.environ:
    ee.Initialize(project=os.environ["EARTHENGINE_PROJECT"], http_transport=httplib2.Http())

else:
    raise ValueError("EARTHENGINE_SERVICE_ACCOUNT or EARTHENGINE_PROJECT environment variable is missing")

[![github](https://img.shields.io/badge/-see%20sources-white?logo=github&labelColor=555)](https://github.com/gee-community/geetools/blob/main/docs/usage/export.ipynb)
[![colab](https://img.shields.io/badge/-open%20in%20colab-blue?logo=googlecolab&labelColor=555)](https://colab.research.google.com/github/gee-community/geetools/blob/main/docs/usage/export.ipynb)

## Example Set up 

Start by defining the image data that will be exported.

In [ ]:
import ee, geetools

In [ ]:
# Load a landsat image and select three bands over the whole mont of january 2023
landsat = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")
    .select(['B4', 'B3', 'B2'])
    .filterDate('2023-01-01', '2023-01-31')
)

# Create a geometry representing an export region.
geometry = ee.Geometry.Rectangle([116.2621, 39.8412, 116.4849, 40.01236])

## to Drive

To export an imageCollection to your Drive account, use `ee.batch.Export.geetools.imagecollection.toDrive()`. For example, to export portions of a Landsat collection, define a region to export, then call Export:

```python 
# Export the image to Cloud Storage.
ee.batch.Export.geetools.imagecollection.toDrive(
  imagecollection = landsat,
  index_property = "system:id",
  description = 'imageCollectionToDriveExample',
  scale = 30,
  region = geometry,
  folder = 'geetools_example',
)
```

When this code is run, a list of export task will be created you will need to `start` them to start the export computation in the server.

## to Cloud Storage

To export an ImageCollection to a Google Cloud Storage bucket, use `ee.batch.Export.geetools.imagecollection.toCloudStorage()`. To export the Landsat image in the previous example to Cloud Storage instead of Drive, use:

```python 
# Export the image to Cloud Storage.
ee.batch.Export.image.toCloudStorage(
  imagecollection =  landsat,
  index_property = "system:id",
  description =  'imageToCloudExample',
  bucket =  'your-bucket-name',
  scale =  30,
  region = geometry
)
```

When this code is run, a list of export task will be created you will need to `start` them to start the export computation in the server.

## To Asset

To export an ImageCollection to an Earth Engine asset, use `ee.batch.Export.geetools.imagecollection.toAsset()`. To export the Landsat image in the previous example to an asset, use:

When this code is run, a list of export task will be created you will need to `start` them to start the export computation in the server.

```python
# Start the export process.
ee.batch.Export.geetools.imagecollection.toAsset(
    imagecollection = landsat,
    index_property = "system:id",
    assetId = 'projects/username/ladnsat_collection',
    scale = 30,
    region = geometry,
    maxPixels = 1e13,
    pyramidingPolicy = {
        'b4': 'mean',
        'b3': 'mean',
        'b2': 'mean'
    }
)
```

For all function please refer to offcial documentation for complete list of parameters of the `ee.batch.Export.image` methods.